### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
nyt_query_url = f"{url}&api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}&fq={filter_query}&sort={sort}&fl={field_list}"


In [4]:
##Test API and understand the results for first page
#test_query_url = f"{nyt_query_url}&page=0"
    
##Make a "GET" request and retrieve the JSON
#test_response = requests.get(test_query_url)
#test_reviews = test_response.json()
#test_reviews

In [ ]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for i in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    query_url = f"{nyt_query_url}&page={i}"
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(query_url)
    reviews = response.json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for doc in reviews["response"]["docs"]:
            reviews_list.append(doc)
        
        # Print the page that was just retrieved
        print(f"Checked page {i}")
    except:
        # Print the page number that had no results then break from the loop
        print(f"Page {i} had no results")
        break


Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6


In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[0:5],indent=4))

In [ ]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df

In [ ]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
reviews_df['title'] = reviews_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
reviews_df

In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords'] = reviews_df['keywords'].apply(extract_keywords)
reviews_df

In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_df['title'].to_list()
titles

### Access The Movie Database API

In [ ]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
details_url = "https://api.themoviedb.org/3/movie/"

tmdb_key_string = "&api_key=" + tmdb_api_key


In [ ]:
##Test Search API call and review results
#test_movie_title = 'The Attachment Diaries'
#test_query_url = f"{url}{test_movie_title}{tmdb_key_string}"

## Test API call and print results
#test_response = requests.get(test_query_url)
#test_movie_info = test_response.json()
#print(json.dumps(test_movie_info, indent=4))

In [ ]:
##Test navigation of Search API results information
#print(f"genre = {test_movie_info['results'][0]['genre_ids']}")
#print(f"lang = {test_movie_info['results'][0]['original_language']}")
#print(f"movie_id = {test_movie_info['results'][0]['id']}")
#test_movie_info['results']

In [ ]:
## Test Details API call and review results
#test_movie_id = 181812   ##Star Wars
#test_detail_query_url = f"{details_url}{test_movie_id}?{tmdb_key_string}"
#test_detail_response = requests.get(test_detail_query_url)
#test_movie_details = test_detail_response.json()
#print(json.dumps(test_movie_details, indent=4))


In [ ]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in titles:
    ##  Test Break after 5 calls to review results
    #if request_counter == 6:
        #break
        
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        print("TMDB Movies List Process is Sleeping")
        time.sleep(1)

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    title_query_url = f"{url}{title}{tmdb_key_string}"
    title_response = requests.get(title_query_url)
    movie = title_response.json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = movie['results'][0]['id']
        #print(f"\nMovie Title: {title}")
        #print(f"Movie Id: {movie_id}")


        # Make a request for a the full movie details
        detail_query_url = f"{details_url}{movie_id}?{tmdb_key_string}"

        
        # Execute "GET" request with url
        detail_response = requests.get(detail_query_url)
        movie_details = detail_response.json()
        #print(movie_details)

        
        # Extract the genre names into a list
        genres_list = []
        for index in range(len(movie_details['genres'])):
            for key in movie_details['genres'][index]:
                if key == 'name':
                    #print(movie_details['genres'][index][key])
                    genres_list.append(movie_details['genres'][index][key])
        #print(f"Genres: {genres_list}")


        # Extract the spoken_languages' English name into a list
        spoken_languages_list = []
        for index in range(len(movie_details['spoken_languages'])):
            for key in movie_details['spoken_languages'][index]:
                if key == 'english_name':
                    #print(movie_details['spoken_languages'][index][key])
                    spoken_languages_list.append(movie_details['spoken_languages'][index][key])
        #print(f"Spoken Language: {spoken_languages_list}")


        # Extract the production_countries' name into a list
        production_countries_list = []
        for index in range(len(movie_details['production_countries'])):
            for key in movie_details['production_countries'][index]:
                if key == 'name':
                    #print(movie_details['production_countries'][index][key])
                    production_countries_list.append(movie_details['production_countries'][index][key])
        #print(f"Production Countries: {production_countries_list}")
        
        
        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append({'title': title,
                                 'original_title' : movie_details['original_title'],
                                 'budget' : movie_details['budget'],
                                 'original_language' : movie_details['original_language'],
                                 'homepage' : movie_details['homepage'],
                                 'overview' : movie_details['overview'],
                                 'popularity' : movie_details['popularity'],
                                 'runtime' : movie_details['runtime'],
                                 'revenue' : movie_details['revenue'],
                                 'release_date' : movie_details['release_date'],
                                 'vote_average' : movie_details['vote_average'],
                                 'vote_count' : movie_details['vote_count'],
                                 'genres' : genres_list,
                                 'spoken_languages' : spoken_languages_list,
                                 'production_countries' : production_countries_list
                                }) 
        print(f"Movie Title: '{title}' -- found")

    except:
        # Print out the title that was found
        print(f"Movie Title: '{title}' -- not found")



In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[0:5],indent=4))

In [ ]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df

### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_nyt_tmdb_df = pd.merge(tmdb_df, reviews_df, how='inner', on='title')
merged_nyt_tmdb_df

In [ ]:
##Test to review 'genres' column before changes
#merged_nyt_tmdb_df['genres']

In [ ]:
##Test to review 'spoken_languages' column before changes
#merged_nyt_tmdb_df['spoken_languages']

In [ ]:
##Test to review 'production_countries' column before changes
#merged_nyt_tmdb_df['production_countries']

In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']
#print(columns_to_fix)

# Create a list of characters to remove
characters_to_remove = ["[","]", "'"]
#print(characters_to_remove)

# Loop through the list of columns to fix
for col in columns_to_fix:
    
    # Convert the column to type 'str'
    merged_nyt_tmdb_df[col] = merged_nyt_tmdb_df[col].astype('str')

    # Loop through characters to remove
    for char in characters_to_remove:
        merged_nyt_tmdb_df[col] = merged_nyt_tmdb_df[col].str.replace(char, "",regex=False)


# Display the fixed DataFrame
merged_nyt_tmdb_df.head()

In [ ]:
##Test to review 'genres' column after changes
#merged_nyt_tmdb_df['genres']

In [ ]:
##Test to review 'spoken_languages' column after changes
#merged_nyt_tmdb_df['spoken_languages']

In [ ]:
##Test to review 'production_countries' column after changes
#merged_nyt_tmdb_df['production_countries']

In [ ]:
# Drop "byline.person" column
merged_nyt_tmdb_df = merged_nyt_tmdb_df.drop(['byline.person'], axis=1)
merged_nyt_tmdb_df

In [ ]:
# Delete duplicate rows and reset index
merged_nyt_tmdb_df = merged_nyt_tmdb_df.drop_duplicates()
merged_nyt_tmdb_df = merged_nyt_tmdb_df.reset_index(drop=True)
merged_nyt_tmdb_df

In [ ]:
# Export data to CSV without the index
merged_nyt_tmdb_df.to_csv('merged_nyt_tmdb_df.csv', encoding='utf-8',index='False')